In [ ]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, decoders
from tokenizers.processors import TemplateProcessing

# 1. Tokenizer Model Oluşturma

In [ ]:
tokenizer = Tokenizer(models.BPE(unk_token="<unk>"))

tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()

tokenizer.decoder = decoders.ByteLevel()

# 2. Tokenizer Ayarlama

In [ ]:
special_tokens = [
    "<pad>",         # Padding token
    "<unk>",         # Bilinmeyen token
    "<s>",           # BOS (beginning of sentence)
    "</s>",          # EOS (end of sentence)
    "<|bos|>",       # GPT-style BOS (CLM için)
    "<|eos|>",       # GPT-style EOS (CLM için)
    "<|endoftext|>", # OpenAI tarzı metin sonu işareti (opsiyonel ama faydalı)
    "<|user|>",      # Kullanıcı prompt başlangıcı
    "<|assistant|>", # Model cevabı başlangıcı
    "<|system|>",    # Sistem talimatları (instruct formatları için)
    "<|sep|>"        # Prompt–response ayracı (isteğe bağlı)
]


trainer = trainers.BpeTrainer(
    vocab_size=50_000, 
    limit_alphabet=2000, 
    max_token_length=100, 
    special_tokens=special_tokens, 
    show_progress=True,
    )

# 3. Tokenizer Eğitimi

In [ ]:
tokenizer.train(files=["./BPE_TokenizerTexts.txt"], trainer=trainer)

In [ ]:
tokenizer.post_processor = TemplateProcessing(
    single="<s> $A </s>",
    pair="<s> $A </s> </s> $B </s>",
    special_tokens=[
        ("<s>", tokenizer.token_to_id("<s>")),
        ("</s>", tokenizer.token_to_id("</s>")),
    ]
)

# 4. Tokenizer Kayıt

In [ ]:
tokenizer.save("tokenizer.json")
print("✅ BPE Tokenizer eğitimi tamamlandı.")




✅ BPE Tokenizer eğitimi tamamlandı.


# 5. Tokenizer Huggingface Formatına Çevir

In [4]:
from transformers import PreTrainedTokenizerFast

hf_tokenizer = PreTrainedTokenizerFast(
    tokenizer_file="./tokenizer.json",
    pad_token="<pad>",
    unk_token="<unk>",
    bos_token="<|bos|>",
    eos_token="<|eos|>",
    additional_special_tokens=[
        "<|user|>", "<|assistant|>", "<|system|>", "<|sep|>", "<|endoftext|>", # "<|bos|>", "<|eos|>", 
    ]
)

hf_tokenizer.save_pretrained("Crispy Tokenizer")
print("✅ HuggingFace uyumlu tokenizer kaydedildi.")


✅ HuggingFace uyumlu tokenizer kaydedildi.
